In [1]:
from datasets import load_dataset, load_from_disk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from minbpe import BasicTokenizer

import torch
from torch import nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import  Dataset, DataLoader

from evaluate import load

## Read Data

In [ ]:
ds = load_dataset("abisee/cnn_dailymail", "3.0.0")

# Save to disk
# ds.save_to_disk("data/cnn_dailymail_dataset")

ds

In [2]:
# If you already save to disk
ds = load_from_disk("data/cnn_dailymail_dataset")
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [3]:
train_data = ds['train'].select_columns(["article", "highlights"])
val_data = ds['validation'].select_columns(["article", "highlights"])
test_data = ds['test'].select_columns(["article", "highlights"])
train_data, val_data, test_data

(Dataset({
     features: ['article', 'highlights'],
     num_rows: 287113
 }),
 Dataset({
     features: ['article', 'highlights'],
     num_rows: 13368
 }),
 Dataset({
     features: ['article', 'highlights'],
     num_rows: 11490
 }))

## Tokenization

### Tokenizer

In [ ]:
# The data used for training
all_articles_text = " ".join(train_data["article"][:1000])
len(all_articles_text)

3530528

In [ ]:
tokenizer = BasicTokenizer()
tokenizer.train(all_articles_text, vocab_size=1024)

In [ ]:
# See how it encode
encoded_article_0 = tokenizer.encode(train_data['article'][0])
encoded_article_0[:5]

[76, 79, 78, 68, 79]

In [26]:
# See how it decode
decoded_article_0 = tokenizer.decode(encoded_article_0)
decoded_article_0, decoded_article_0 == train_data['article'][0]

('LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details 

In [ ]:
# Add special tokens, we do not need <unk> here because data is in english and fit in ASCII
max_vocab_id = list(tokenizer.vocab.keys())[-1]
tokenizer.special_tokens = {
    "<sos>": max_vocab_id + 1,
    "<eos>": max_vocab_id + 2,
    "<unk>": max_vocab_id + 3,
    "<pad>": max_vocab_id + 4,
}

In [ ]:
# See how it works with special tokens
encoded_special_0 = tokenizer.encode(f"<sos> {train_data['article'][0]} <eos> <pad> <pad>")
decoded_special_0 = tokenizer.decode(encoded_special_0)
decoded_special_0, decoded_special_0 == f"<sos> {train_data['article'][0]} <eos> <pad> <pad>"

('<sos> LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. De

In [30]:
# Save to disk
tokenizer.save("model/model_article_1000")

In [2]:
# Load from disk
tokenizer = BasicTokenizer()
tokenizer.load("model/model_article_1000.model")

### Tokenize Data

In [ ]:
def tokenize_fields(example):
    example["article"] = tokenizer.encode(example["article"])
    example["highlights"] = tokenizer.encode(example["highlights"])
    return example

# Tokenize each split
train_tokenized = train_data.select(range(10_000)).map(tokenize_fields)
val_tokenized = val_data.select(range(2500)).map(tokenize_fields)
test_tokenized = test_data.select(range(2500)).map(tokenize_fields)

In [ ]:
# Save to disk
train_tokenized.save_to_disk("data/cnn_train_tokenized_10k")
val_tokenized.save_to_disk("data/cnn_val_tokenized_2500")
test_tokenized.save_to_disk("data/cnn_test_tokenized_2500")

In [3]:
# If you already save to disk
train_tokenized = load_from_disk("data/cnn_train_tokenized_10k")
val_tokenized = load_from_disk("data/cnn_val_tokenized_2500")
test_tokenized = load_from_disk("data/cnn_test_tokenized_2500")

## Vectorizer

In [4]:
class Vectorizer:
  def __init__(self, tokenizer: BasicTokenizer):
    self.tokenizer = tokenizer
    self.vocab_size = len(tokenizer.vocab)
    self.sos_idx = tokenizer.special_tokens["<sos>"]
    self.eos_idx = tokenizer.special_tokens["<eos>"]
    self.pad_idx = tokenizer.special_tokens["<pad>"]

  # Add <sos> and <eos> at the start and end, pad data if necessasry
  def index_vectorize(self, tokens, max_length=1024):
    indices = tokens[:max_length - 2]
    indices = [self.sos_idx] + indices + [self.eos_idx]
    indices += [self.pad_idx] * (max_length - len(indices))
    return indices

In [5]:
article_vectorizer = Vectorizer(tokenizer)

In [61]:
tokenizer.decode(article_vectorizer.index_vectorize(encoded_article_0))

'<sos>LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Deta

## Dataset

In [6]:
class IndexArticleDataset(Dataset):
  def __init__(self, input, target, vectorizer: Vectorizer, max_input_length=1024, max_target_length=128):
    self.input = input
    self.target = target
    self.vectorizer = vectorizer
    
    self.max_input_length = max_input_length
    self.max_target_length = max_target_length
    
    self.sos_index = vectorizer.sos_idx
    self.eos_index = vectorizer.eos_idx
    self.pad_index = vectorizer.pad_idx

  def __len__(self):
    return len(self.input)

  def __getitem__(self, index):
    return {'x': torch.as_tensor(self.vectorizer.index_vectorize(self.input[index], self.max_input_length)),
            'y': torch.as_tensor(self.vectorizer.index_vectorize(self.target[index], self.max_target_length))}

  def get_vectorizer(self):
    return self.vectorizer 
  
  def get_num_batches(self, batch_size):
    return len(self) // batch_size 

In [7]:
train_dataset = IndexArticleDataset(train_tokenized['article'], 
                                    train_tokenized['highlights'], 
                                    article_vectorizer,
                                    max_input_length=512,
                                    max_target_length=128)

val_dataset = IndexArticleDataset(val_tokenized['article'], 
                                  val_tokenized['highlights'], 
                                  article_vectorizer,
                                  max_input_length=512,
                                  max_target_length=128)

test_dataset = IndexArticleDataset(test_tokenized['article'], 
                                   test_tokenized['highlights'], 
                                   article_vectorizer,
                                   max_input_length=512,
                                   max_target_length=128)

In [85]:
out_dict = train_dataset.__getitem__(0)
len(out_dict['x']), len(out_dict['y'])

(512, 128)

In [87]:
tokenizer.decode(out_dict['x'].numpy()), tokenizer.decode(out_dict['y'].numpy())

('<sos>LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Det

## Dataloader

In [8]:
def generate_batches(dataset, batch_size, 
                     shuffle=True,
                     drop_last=True, 
                     device="cpu"):
  
  dataloader = DataLoader(dataset=dataset, 
                          batch_size=batch_size,
                          shuffle=shuffle, 
                          drop_last=drop_last)

  for data_dict in dataloader:
    out_data_dict = {}
    for name, tensor in data_dict.items():
      out_data_dict[name] = data_dict[name].to(device)
    yield out_data_dict

In [91]:
# Using generate_batches to get 1 sample at a time
for out_dict in generate_batches(dataset=train_dataset, batch_size=32):
  x_data = out_dict['x']
  y_target = out_dict['y']
  print(f"x shape: {x_data.shape}, y shape: {y_target.shape}")
  print(x_data)
  print(y_target)
  break

x shape: torch.Size([32, 512]), y shape: torch.Size([32, 128])
tensor([[1024,   40,   69,  ...,   78,  900, 1025],
        [1024,   40,   84,  ...,  401,  276, 1025],
        [1024,   78,   69,  ..., 1027, 1027, 1027],
        ...,
        [1024,   76,   79,  ...,  735,  373, 1025],
        [1024,  702,   65,  ...,  456,  412, 1025],
        [1024,  651,   32,  ...,  926,  549, 1025]])
tensor([[1024,   69,  115,  ..., 1027, 1027, 1027],
        [1024,   65,   99,  ..., 1027, 1027, 1027],
        [1024,   80,  304,  ...,  331,   67, 1025],
        ...,
        [1024,   82,  265,  ..., 1027, 1027, 1027],
        [1024,   78,   69,  ..., 1027, 1027, 1027],
        [1024,   75,  612,  ...,  334,  473, 1025]])


## Model & Training

In [ ]:
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
class Head(nn.Module):
  def __init__(self, embed_dim, head_dim, max_input_length, dropout=0.5, is_masked=True):
    super().__init__()
    self.query = nn.Linear(embed_dim, head_dim, bias=False)
    self.key = nn.Linear(embed_dim, head_dim, bias=False)
    self.value = nn.Linear(embed_dim, head_dim, bias=False)
    self.is_masked = is_masked
    if is_masked:
      self.register_buffer('mask', torch.tril(torch.ones(max_input_length, max_input_length)))
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, x, context=None):
    B, T, C = x.shape # (batch_size, time_steps, channels=embed_dim)

    if context == None:
      context = x

    Q = self.query(x)
    K = self.key(context)

    KQ = (Q @ K.transpose(-2, -1)) 
    KQ_scaled = KQ / (np.sqrt(K.shape[-1]))
    if self.is_masked:
      KQ_scaled = KQ_scaled.masked_fill(self.mask[:T, :T] == 0, float('-inf'))
    KQ_normalized_masked_softmaxed = torch.softmax(KQ_scaled, dim=-1)
    KQ_normalized_masked_softmaxed_dropouted = self.dropout(KQ_normalized_masked_softmaxed)

    V = self.value(context)
    out = KQ_normalized_masked_softmaxed_dropouted @ V
    return out
  
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, embed_dim, max_input_length, dropout=0.5, is_masked=True):
    super().__init__()
    head_dim = embed_dim // num_heads
    self.heads = nn.ModuleList([Head(embed_dim, head_dim, max_input_length, dropout, is_masked) for _ in range(num_heads)])
    self.linear = nn.Linear(head_dim * num_heads, embed_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, k, context=None):
    out = torch.cat([h(k, context) for h in self.heads], dim=-1)
    out = self.linear(out)
    out = self.dropout(out)
    return out
  
class FeedFoward(nn.Module):
  def __init__(self, embed_dim, dropout=0.5):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(embed_dim, 4*embed_dim), # default: embed_dim=512, feed_forward_dim=2048)
      nn.ReLU(),
      nn.Linear(4*embed_dim, embed_dim),
      nn.Dropout(dropout),
    )

  def forward(self, x, context=None):
    return self.layers(x)
  
class SubLayer(nn.Module):
  def __init__(self, layer: nn.Module, embed_dim, dropout=0.5):
    super().__init__()
    self.Layer = layer
    self.ln = nn.LayerNorm(embed_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, context=None):
    out = self.Layer(x, context)
    out = self.dropout(out)
    out = self.ln(x + out) # normalization after residual connection
    return out
  
class Encoder(nn.Module):
  def __init__(self, num_heads, embed_dim, max_input_length, dropout=0.5):
    super().__init__()
    self.multi_head_attention = SubLayer(MultiHeadAttention(num_heads, embed_dim, max_input_length, dropout, is_masked=False), embed_dim, dropout)
    self.feed_forward = SubLayer(FeedFoward(embed_dim, dropout), embed_dim, dropout)

  def forward(self, x):
    out = self.multi_head_attention(x)
    out = self.feed_forward(out)
    return out
  
class Decoder(nn.Module):
  def __init__(self, num_heads, embed_dim, max_input_length, dropout=0.5):
    super().__init__()
    self.masked_multi_head_attention = SubLayer(MultiHeadAttention(num_heads, embed_dim, max_input_length, dropout, is_masked=True), embed_dim, dropout)
    self.unmasked_multi_head_attention = SubLayer(MultiHeadAttention(num_heads, embed_dim, max_input_length, dropout, is_masked=False), embed_dim, dropout)
    self.feed_forward = SubLayer(FeedFoward(embed_dim, dropout), embed_dim, dropout)

  def forward(self, x, context=None):
    out = self.masked_multi_head_attention(x)
    
    # In Encoder-Decoder model, context is Encoder output
    # In Decoder-only model, context is None and skip this process
    if context is not None:
      out = self.unmasked_multi_head_attention(out, context)
    
    out = self.feed_forward(out)
    return out

class PositionalEncoding(nn.Module):
  def __init__(self, embed_dim, max_input_length):
    super().__init__()

    # (max_input_length, 1)
    pos = torch.arange(0, max_input_length, dtype=torch.float).unsqueeze(1)
    
    # (embed_dim/2,)
    i = torch.arange(0, embed_dim, 2).float()
    div_term = torch.exp(i * (-np.log(10000.0) / embed_dim))

    # (max_input_length, embed_dim/2)
    rate = pos * div_term

    pe = torch.zeros(max_input_length, embed_dim)
    pe[:, 0::2] = torch.sin(rate)  # even indices
    pe[:, 1::2] = torch.cos(rate)  # odd indices

    pe = pe.unsqueeze(0)  # (1, T, D) - for broadcasting over batch
    self.register_buffer('pe', pe)

  def forward(self, x):
    B, T = x.shape
    return self.pe[:, :T, :]
  
# Decoder only model
class Transformer_Decoder(nn.Module):
  def __init__(self, num_layers, vocab_size, num_heads, embed_dim, max_input_length, dropout=0.5):
    super().__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim)
    self.pos_encoder = PositionalEncoding(embed_dim, max_input_length)
    self.dropout = nn.Dropout(dropout)
    self.layers = nn.Sequential(
      *[Decoder(num_heads, embed_dim, max_input_length, dropout) for _ in range(num_layers)],
      nn.Linear(embed_dim, vocab_size),
      )

  def forward(self, x):
    embed_x = self.embed(x)
    pos_embed_x = self.pos_encoder(x)
    embed_input = embed_x + pos_embed_x
    embed_input = self.dropout(embed_input)
    out = self.layers(embed_input)
    return out

# Encoder-Decoder model
class Transformer_Encoder_Decoder(nn.Module):
  def __init__(self, vocab_size, num_layers, num_heads, embed_dim, max_input_length, max_target_length, pad_idx, dropout=0.5):
    super().__init__()
    self.input_embed = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
    self.input_pos_encoder = PositionalEncoding(embed_dim, max_input_length)
    self.input_dropout = nn.Dropout(dropout)
    
    self.target_embed = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
    self.target_pos_encoder = PositionalEncoding(embed_dim, max_target_length)
    self.target_dropout = nn.Dropout(dropout)
    
    self.encoder = nn.ModuleList([Encoder(num_heads, embed_dim, max_input_length, dropout) for _ in range(num_layers)])
    self.decoder = nn.ModuleList([Decoder(num_heads, embed_dim, max_input_length, dropout) for _ in range(num_layers)])
    self.linear = nn.Linear(embed_dim, vocab_size)

  def forward(self, x, y):
    embed_x = self.input_embed(x)
    pos_embed_x = self.input_pos_encoder(x)
    embed_input = embed_x + pos_embed_x
    embed_input = self.input_dropout(embed_input)
    
    embed_y = self.target_embed(y)
    pos_embed_y = self.target_pos_encoder(y)
    embed_target = embed_y + pos_embed_y
    embed_target = self.target_dropout(embed_target)

    for layer in self.encoder:
      embed_input = layer(embed_input)
    encoder_out = embed_input

    for layer in self.decoder:
      embed_target = layer(embed_target, encoder_out)
    decoder_out = embed_target

    out = self.linear(decoder_out)
    return out

In [ ]:
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

max_input_length = 512
max_target_length = 128

vocab_size = len(tokenizer.vocab)
pad_idx = tokenizer.special_tokens['<pad>']

num_layers = 6
num_heads = 8
embed_dim = 512
input_dropout = 0.1

lr = 0.001
epochs = 100
eval_interval = 1
batch_size = 32
batch_number_train = len(train_dataset) // batch_size
batch_number_val = len(val_dataset) // batch_size

model = Transformer_Encoder_Decoder(vocab_size, num_layers, num_heads, embed_dim, max_input_length, max_target_length, pad_idx, input_dropout).to(device)
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for epoch in range(epochs):
  model.train()

  train_loss = 0

  for out_dict in generate_batches(dataset=train_dataset, batch_size=batch_size, device=device):
    x = out_dict['x']
    y = out_dict['y']

    y_logits = model(x, y[:, :-1])
    
    B, T, C = y_logits.shape
    loss = loss = loss_fn(y_logits.reshape(B * T, C), y[:, 1:].reshape(B * T))
    train_loss += loss.item()

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  train_loss /= batch_number_train

  if epoch % eval_interval == 0 or epoch == epochs-1:
    model.eval()
    val_loss = 0

    with torch.inference_mode():
      for out_dict in generate_batches(dataset=val_dataset, batch_size=batch_size):
        x = out_dict['x']
        y = out_dict['y']

        y_logits = model(x, y[:, :-1])
    
        B, T, C = y_logits.shape
        loss = loss_fn(y_logits.reshape(B * T, C), y[:, 1:].reshape(B * T))
        val_loss += loss.item()

    val_loss /= batch_number_val
    
    print(f"Epoch {epoch} | Train loss: {train_loss:.4f} | Val loss: {val_loss:.4f}")